# Logging 包基础教程

所谓日志就是一种在软件运行时能够记录事件的方法，可以在特定的位置写上代码从而能够确认代码有没有执行到这个点。  
一般来说最开始大家都只会使用 print(),但是print()的缺点也显而易见
* 程序结束关掉终端就再也没有了
* 无法保存到本地
* 不够灵活，有些信息在测试环境可能需要体现，但是在生产环境是不需要体现的（当然上生产了肯定不会用print()）

本教程是最基本的教程，顺序按照自己使用的频率做了调整，其中可能比较难理解的概念如下：
* basicConfig: 一个一次性简单配置文件
* logging 本身是"全局"的，多模块调用的时候，只要import logging 就好

## 何时应用logging

|场景|推荐应用方法|
|:---|:---|
|在终端执行程序或脚本调试|print( )|
|报告事件在程序执行期间发生（用于监控和错误追踪）|logging.info( ),logging.debug( )|
|警报|如果这个warning是要提醒用户去删除的，则用 warnings.warn( )<br\>如果仅仅是要提醒，不需要客户端操作什么，使用 logging.warning( )|
|报告错误|使用 raise 就可以|
|报告未知或已知错误，且不中断程序运行|logging.error( ), logging.exception( ), logging.critical( )|

## 各种等级描述

|等级|何时使用|
|:---|:---|
|DEBUG|细节问题，通常仅仅用在debug的时候|
|INFO|确认事件正在往期望的方向发展|
|WARNING|程序仍然运行，但是中间出现了一点和期望不相符的事件|
|ERROR|严重的错误导致程序无法完成某个功能|
|CRITICAL|严重错误导致整个程序无法继续运行|

默认的等级是 WARNING，指的是只有wanrning以上的信息，才会被记录下来，等级比他低的消息是不会记录的。  
所谓记录，可以有多种方法，最简单的是将记录打印在终端，另外一种方法是存在某个文件中。_个人习惯的方法是按照天存在文件中，到10天后删除_

## 简单示例

In [1]:
import logging
logging.warning('Watch out!') # 这条语句将在终端商打印一条信息
logging.info('I told you so')  # 这个就没啥用，因为现在的level是 warning

有关格式，以及这条wangning是谁发出的之类的问题，在之后会介绍

## 有关basicConfig

* basicConfig 是一个一次性调用工具，只有第一次调用的时候才有意义，之后所有的调用都没有意义，这也是为什么需要重启解释器的原因
* basicConfig 应该在所有 debug，info等发生前发生
* 如果在使用 debug，info等信息记录功能前，没有显式的调用 basicConfig，则其实内部默认运行了 basicConfig，所以之后再写也是没有用的

## 日志文件

In [1]:
# 运行本段代码需要重启解释器
import logging
logging.basicConfig(filename='example.log',level=logging.DEBUG)
logging.debug('This message should go to the log file')
logging.info('So should this')
logging.warning('And this, too')

这是 example.log 中会显示  
>DEBUG:root:This message should go to the log file  
>INFO:root:So should this  
>WARNING:root:And this, too  

一般来讲日志文件都是采用 append 的行为，但是如果想清空之前的日志文件，仅记录本次的记录也可以

In [1]:
# 运行本段代码需要重启解释器
# filemode = 'w'
import logging
logging.basicConfig(filename='example.log',level=logging.DEBUG, filemode='w')
logging.debug('This message should go to the log file')
logging.info('So should this')
logging.warning('And this, too')

## 往日志记录传入变量

In [1]:
# 需要重启解释器
import logging
logging.warning('%s before you %s', 'Look', 'leap!')

当然用普通的format的形式也可以，之所以使用 %s 这种形式是为了做好兼容性，新的格式也是支持的，可以阅读 [这里 ](https://docs.python.org/3/howto/logging-cookbook.html#formatting-styles)解锁更高级的技能

In [2]:
logging.warning('%s before you %s'%('Look','leap!'))
logging.warning('{0} before you {1}'.format('Look','leap!'))

## 修改日志记录格式

需要确定好你想要什么格式来输出日志信息

In [1]:
# 需要重启解释器
import logging
logging.basicConfig(format='%(asctime)s|%(levelname)s|%(name)s|%(message)s', level=logging.DEBUG)
logging.debug('This message should appear on the console')
logging.info('So should this')
logging.warning('And this, too')

2019-01-29 17:22:40,450|DEBUG|root|This message should appear on the console
2019-01-29 17:22:40,452|INFO|root|So should this
2019-01-29 17:22:40,454|WARNING|root|And this, too


修改格式最麻烦的在于记不住对应的简写，链接在[这边](https://docs.python.org/3/library/logging.html#logrecord-attributes)  
不过其实也就只需要知道几个常用变量：
* levelname: %(levelname)s 等级（字符串）
* message: %(message)s 信息
* asctime: %(asctime)s 人类可读时间格式
* name: %(name)s 日志记录器的名字，这个其实就是之前所说的 source，日志记录器其实可以有多个，按照名字做区分，名字可以是脚本文件的 \_\_name\_\_

特殊一点的是时间 asctime ，因为是可以有多种表达格式的，特地多了一个datefmt的参数

In [1]:
# 需要重启解释器
import logging
# datefmt 的格式参考 strftime
logging.basicConfig(format='%(asctime)s|%(levelname)s|%(name)s|%(message)s', level=logging.DEBUG, datefmt='%m/%d/%Y %I:%M:%S %p')
logging.debug('This message should appear on the console')
logging.info('So should this')
logging.warning('And this, too')

01/29/2019 05:26:27 PM|DEBUG|root|This message should appear on the console
01/29/2019 05:26:27 PM|INFO|root|So should this
01/29/2019 05:26:27 PM|WARNING|root|And this, too


## 多模块日志记录

下面介绍一种简单的多文件的日志记录方法。   
本质是只要在主文件调用一次 logging.basicConfig() 就可以，其余所有文件只需要调用 logging 就可以直接用

In [1]:
# 需要重启解释器
# myapp.py
import logging
import mylib

def main():
    logging.basicConfig(level=logging.INFO)
    logging.info('Started')
    mylib.do_something()
    logging.info('Finished')

if __name__ == '__main__':
    main()

INFO:root:Started
INFO:root:Doing something
INFO:root:Finished


可以看到在别的模板上的 Doing something 日志也成功打印了  
但是你会发现这里显示的source 是 root，也就是没有标记出这个日志的来源是 mylib  
如果想要标记日志记录的来源，需要看 AdvancedLoggingTutorial

## 命令行控制日志等级

之后你可能会想在输入命令行时使用 --log=INFO 来决定本次运行脚本的日志等级。   
* 获取变量，假设叫做 loglevel
* 利用 getattr(logging, loglevel.upper()) 获得相应的数值等级
* 将数值等级传入 basicConfig

In [1]:
# 运行本段代码需要重启解释器
import logging
# 假设这个loglevel是外部脚本获取的
loglevel = 'info' 
# 获得数字等级
numeric_level = getattr(logging, loglevel.upper(), None)
if not isinstance(numeric_level, int):
    raise ValueError('Invalid log level: %s' % loglevel)
logging.basicConfig(level=numeric_level)

logging.debug('This is a debug message')
logging.info('This is an info message')

INFO:root:This is an info message
